<a href="https://colab.research.google.com/github/thejatinbabbar/keras-models/blob/master/streamlit_keras_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import subprocess

try:
    import streamlit
    print('Streamlit imported')
except ModuleNotFoundError:
    try:
        out = subprocess.check_output([sys.executable, '-m', 'pip', 'install', 'streamlit'])
        print(out)
    except Exception as e:
        print(e)

try:
    from pyngrok import ngrok
    print('Pyngrok imported')
except ModuleNotFoundError:
    try:
        out = subprocess.check_output([sys.executable, '-m', 'pip', 'install', 'pyngrok'])
        print(out)
        from pyngrok import ngrok
    except Exception as e:
        print(e)

Streamlit imported
Pyngrok imported


In [2]:
%%writefile app.py
import streamlit as st
import pandas as pd
from tensorflow import keras

st.title('Keras MNIST')

# print(keras.__version__)

st.header('Dataset')
df_train = pd.read_csv('/content/sample_data/mnist_train_small.csv', header=None)
df_test = pd.read_csv('/content/sample_data/mnist_test.csv', header=None)

# df_train.head()
st.write('DF Train')
st.write(df_train)

# df_test.head()
st.write('DF Test')
st.write(df_test)

st.write(df_train.info())

st.write(df_test.info())

x_train, y_train = df_train.loc[:, 1:], df_train.loc[:, 0]
x_test, y_test = df_test.loc[:, 1:], df_test.loc[:, 0]

def normalize(feat):
    feat = feat / 255
    return feat

x_train = normalize(x_train)
x_test = normalize(x_test)

y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=101)

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input

model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(784,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', metrics='accuracy', loss='categorical_crossentropy')

model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_val, y_val))

model.evaluate(x_test, y_test)

Overwriting app.py


In [3]:
public_url = ngrok.connect(port='8501')

2020-10-05 10:24:08.938 INFO    pyngrok.process: ngrok process starting: 231
2020-10-05 10:24:08.953 INFO    pyngrok.process: t=2020-10-05T10:24:08+0000 lvl=info msg="no configuration paths supplied"
2020-10-05 10:24:08.957 INFO    pyngrok.process: t=2020-10-05T10:24:08+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
2020-10-05 10:24:08.958 INFO    pyngrok.process: t=2020-10-05T10:24:08+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil
2020-10-05 10:24:08.961 INFO    pyngrok.process: t=2020-10-05T10:24:08+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040
2020-10-05 10:24:09.757 INFO    pyngrok.process: t=2020-10-05T10:24:09+0000 lvl=info msg="tunnel session started" obj=tunnels.session
2020-10-05 10:24:09.758 INFO    pyngrok.process: t=2020-10-05T10:24:09+0000 lvl=info msg="client session established" obj=csess id=87d01cc05248
2020-10-05 10:24:09.763 INFO    pyngrok.process: ngrok process has started

In [4]:
public_url

'http://d499191a9de9.ngrok.io'

In [5]:
!streamlit run app.py &>/dev/null&